# Analysis of survey results

In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('survey.db', check_same_thread=False)

df_new = pd.read_sql_query("SELECT * FROM responses", conn)
print(df_new.shape)
df_new.head()

(514, 8)


,user_id,question_num,cleaned_index,gen_index,gen_column,user_choice,is_correct,timestamp
0,524858799,1,135,23,gen_yagpt_8b,2,1,2025-04-27 13:23:09
1,524858799,2,71,197,gen_rugpt_13b,1,0,2025-04-27 13:23:22
2,524858799,3,5,112,gen_tliteit_7b,2,1,2025-04-27 13:23:25
3,524858799,4,166,42,gen_llama3_8b_lora,2,0,2025-04-27 13:23:29
4,524858799,5,45,76,gen_mistral_7b_lora,1,0,2025-04-27 13:23:31


fraction of correct answers

In [7]:
df_new["is_correct"].sum() / df_new.shape[0]

0.603112840466926

count of people

In [15]:
questions_count = 7
df_new.shape[0] / questions_count

73.42857142857143

In [9]:
import numpy as np

sub_df = df_new.groupby('gen_column').agg({
                         'is_correct': [np.sum, 'count'],
                         })
sub_df[('is_correct',   'frac_found')] = sub_df[('is_correct',   'sum')] / sub_df[('is_correct',   'count')]
sub_df.columns = [col[0] + "_" + col[1] for col in sub_df.columns.values]
sub_df = sub_df.reset_index()

In [10]:
sub_df

,gen_column,is_correct_sum,is_correct_count,is_correct_frac_found
0,gen_llama3_8b_lora,39,73,0.534247
1,gen_mistral_7b_lora,37,71,0.521127
2,gen_qwen_7b,50,69,0.724638
3,gen_qwen_7b_lora,44,69,0.637681
4,gen_rugpt_13b,43,77,0.558442
5,gen_tliteit_7b,50,75,0.666667
6,gen_yagpt_8b,47,80,0.587500


In [11]:
from scipy.stats import binomtest

pvals = []
for row in range(sub_df.shape[0]):
    k, n = sub_df.iloc[row]["is_correct_sum"], sub_df.iloc[row]["is_correct_count"]
    res = binomtest(k=k, n=n)
    pvals.append(np.round(res.pvalue, 5))

pvals

[0.63997, 0.81259, 0.00024, 0.02949, 0.36203, 0.00523, 0.14564]

In [12]:
sub_df["p-value"] = pvals

In [14]:
renamed_df = sub_df.rename(columns={
    "gen_column": "модель", 
    "is_correct_sum": "кол-во правильных", 
    "is_correct_count": "кол-во оценок",
    "is_correct_frac_found": "доля правильных"
    })
renamed_df

,модель,кол-во правильных,кол-во оценок,доля правильных,p-value
0,gen_llama3_8b_lora,39,73,0.534247,0.63997
1,gen_mistral_7b_lora,37,71,0.521127,0.81259
2,gen_qwen_7b,50,69,0.724638,0.00024
3,gen_qwen_7b_lora,44,69,0.637681,0.02949
4,gen_rugpt_13b,43,77,0.558442,0.36203
5,gen_tliteit_7b,50,75,0.666667,0.00523
6,gen_yagpt_8b,47,80,0.587500,0.14564


In [ ]:
renamed_df.to_csv("result.csv", index=False)
renamed_df.to_excel("result.xlsx", index=False)